In [12]:
# If missing requirements: !pip install geopandas opera-utils pyarrow seaborn

In [1]:
import folium
import geopandas as gpd
import opera_utils
import pandas as pd

In [2]:
df = pd.read_parquet("disp-s1-unwrapping-runtimes.parquet")

# Convert to hours
runtime_cols = [
    "runtime_wrapped_phase",
    "runtime_stitching_bursts",
    "runtime_unwrap",
    "runtime_timeseries_inversion",
    "runtime_dolphin",
    "runtime_product",
    "runtime_disp_s1",
]
for col in runtime_cols:
    df[col] = (df[col] / 3600)
# Each ministack has 42 ifgs (except for first, which does 39)
# We run a default of n_parallel_jobs = 4 , factor back for the per-ifg number
# https://github.com/opera-adt/disp-s1/blob/d6e27cc06bbeda8ca1c3a0a7c98b351597e1d825/configs/algorithm_parameters_historical_20250401.yaml#L87
df["runtime_average_per_ifg_minutes"] = df["runtime_unwrap"] * 60 / 42 * 4


# Add the DISP-S1 Frame geometries
gdf = gpd.GeoDataFrame(
    pd.merge(
        df, opera_utils.get_frame_geodataframe(), left_on="frame_id", right_index=True
    )
)

In [3]:
gdf.iloc[0].T

frame_id                                                                        8882
product_version                                                                  0.8
generation_datetime                                              2024-11-14 23:56:30
burst_id_count                                                                    27
unwrap_method                                                                 snaphu
max_memory_gb                                                                  34.18
venue                                                                            pst
logname                            OPERA_L3_DISP-S1_IW_F08882_v0.8_20241114T23563...
runtime_wrapped_phase                                                            NaN
runtime_stitching_bursts                                                         NaN
runtime_unwrap                                                                   NaN
runtime_snaphu                                                   

In [4]:
def explore(gdf, column, cmap="RdBu_r", vmax=240, vmin=30):  # noqa: D103
    m = folium.Map()

    # Create separate layers for each orbit pass type
    ascending = gdf[gdf["orbit_pass"] == "ASCENDING"].copy()
    descending = gdf[gdf["orbit_pass"] == "DESCENDING"].copy()
    # Sort by longitude (west to east) so eastern frames are on top
    # Use the centroid of each geometry to determine its longitude
    ascending.loc[:, "center_x"] = ascending.geometry.to_crs("EPSG:3857").centroid.x
    ascending = ascending.sort_values(by="center_x")
    descending.loc[:, "center_x"] = descending.geometry.to_crs("EPSG:3857").centroid.x
    descending = descending.sort_values(by="center_x")

    # Add layers with different colors
    ascending.explore(
        m=m,
        cmap=cmap,
        column=column,
        name="ASCENDING",
        vmin=vmin,
        vmax=vmax,
        style_kwds={"fillOpacity": 0.7},
    )

    descending.explore(
        m=m,
        cmap=cmap,
        column=column,
        name="DESCENDING",
        vmin=vmin,
        vmax=vmax,
        style_kwds={"fillOpacity": 0.7},
        legend=False,
    )

    # Add layer control
    folium.LayerControl().add_to(m)
    return m

def explore_column(  # noqa: D103
    gdf, col_name: str, agg_func="mean", cmap="RdYlBu_r", vmin_pct=.20, vmax_pct=0.95
) -> folium.Map:
    out = gdf.groupby("frame_id")[["geometry", "orbit_pass", col_name]].agg(
        {"geometry": "first", "orbit_pass": "first", col_name: agg_func}
    )
    out = out.set_geometry("geometry").set_crs(gdf.crs)
    vmin = out[col_name].quantile(vmin_pct)
    vmax = out[col_name].quantile(vmax_pct)
    return explore(out, col_name, cmap=cmap, vmax=vmax, vmin=vmin)

In [5]:
gdf_snaphu_ops = gdf[
    (gdf.unwrap_method == "snaphu") & (gdf.venue == "ops")
]

In [6]:
gdf_snaphu_ops.describe()

,frame_id,product_version,generation_datetime,burst_id_count,max_memory_gb,runtime_wrapped_phase,runtime_stitching_bursts,runtime_unwrap,runtime_timeseries_inversion,runtime_dolphin,runtime_product,runtime_disp_s1,spurt_total_num_points,runtime_average_per_ifg_minutes,is_land
count,10607.000000,10607.0,10607,10607.000000,10607.000000,10607.000000,10607.000000,10607.000000,10607.000000,10607.000000,10607.000000,10607.000000,0.0,10607.000000,10607.0
mean,23181.302630,1.0,2025-07-22 01:01:02.425379,21.566701,35.991921,0.297724,0.130170,4.824759,0.167255,5.941498,0.524575,6.466073,NaN,27.570051,1.0
min,831.000000,1.0,2025-04-13 17:37:06,1.000000,28.990000,0.070206,0.032792,0.792825,0.050000,1.004669,0.203106,1.218072,NaN,4.530429,1.0
25%,12637.000000,1.0,2025-07-25 04:34:24,18.000000,34.210000,0.265975,0.110410,3.956140,0.159167,4.816781,0.430706,5.258542,NaN,22.606516,1.0
50%,22922.000000,1.0,2025-08-09 05:05:07,27.000000,35.890000,0.311719,0.145542,5.021364,0.172778,6.332739,0.563858,6.911647,NaN,28.693508,1.0
75%,34472.000000,1.0,2025-08-30 14:08:54.500000,27.000000,37.570000,0.345061,0.156851,5.698139,0.185833,7.052736,0.621174,7.649587,NaN,32.560794,1.0
max,46543.000000,1.0,2025-09-04 09:04:09,27.000000,46.100000,0.461633,0.179386,13.177086,0.262222,14.580717,0.742028,15.157728,NaN,75.297635,1.0
std,13209.056699,0.0,NaN,7.941046,2.357158,0.064024,0.034629,1.488676,0.041679,1.780510,0.116437,1.872265,NaN,8.506719,0.0


In [11]:
explore_column(
    gdf_snaphu_ops,
    "runtime_disp_s1",
    vmax_pct=.95,
    agg_func='mean',
)

In [12]:
explore_column(
    gdf_snaphu_ops,
    "runtime_wrapped_phase",
)

In [13]:
explore_column(
    gdf_snaphu_ops,
    "runtime_unwrap",
)

In [14]:
explore_column(
    gdf_snaphu_ops,
    "runtime_average_per_ifg_minutes",
    agg_func="median",
)